## TYTANチュートリアル（巡回セールスマン問題）

2023年4月20日

ビネクラ安田

出典：[量子アニーリング（D-wave）で巡回セールスマン問題を解く](https://vigne-cla.com/21-8/)

### 問題
4つの都市A, B, C, Dを最短で訪れる経路を求める。スタートの都市は決まってない。各都市を結ぶ道の距離は分かっている。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_1.png" width = 40%>
</div>

###エンコーディング（≒量子ビットで表現できるように変換する）

4×4のテーブルを作り16個の量子ビットを対応させる。これはTSPの定石なので覚える。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_2.png" width = 50%>
</div>

### QUBOモデルでは何が設定できるか？（おさらい）

**<font color="red">「n個の量子ビットからm個を1にする」</font>**

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

**<font color="red">「2個の量子ビットが同時に1になったらペナルティを与える」</font>**

例）2個の量子ビットが同時に1になったらペナルティ0.5
```
H = 0.5 * (q1 * q2)
```

今回はこれらだけを使うが、その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件（絶対に守ってほしい条件）

各行、各列の量ビットは1個だけ1になってほしい。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_3.png" width = 45%>
</div>


```
#1番目に訪れる都市は1つだけにしたい、のやつ
H = (q00 + q01 + q02 + q03 - 1)**2
H = (q04 + q05 + q06 + q07 - 1)**2
H = (q08 + q09 + q10 + q11 - 1)**2
H = (q12 + q13 + q14 + q15 - 1)**2

#都市Aに訪れる順番は1つだけにしたい、のやつ
H = (q00 + q04 + q08 + q12 - 1)**2
H = (q01 + q05 + q09 + q13 - 1)**2
H = (q02 + q06 + q10 + q14 - 1)**2
H = (q03 + q07 + q11 + q15 - 1)**2
```


### コスト（できるだけ小さくしたい≒ペナルティ）

都市間の距離に基づいた設定を行う。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_4.png" width = 70%>
</div>

赤矢印は1番目の都市から2番目の都市Aに行く4パターン。例えば、[q1, q4] が同時に1になることは、1番目(B)→2番目(A)を訪れることを表す。B→Aの距離3に比例したコスト（ペナルティ）（ただし制約条件よりも規模を小さくするため10分の1して0.3）を与える。

```
H = 0.3 * (q01 * q04)
※全部で48式
```

式より、[q1, q4] が同時に１になるとエネルギーに0.3が追加される。距離に比例したコスト（ペナルティ）を設定することで、距離が長い道ほど採用しにくくなり、結果として最短経路が求まる。


## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan
!pip install pyqubo

In [ ]:
from tytan import *
import numpy as np
from pyqubo import Binary

#量子ビットを用意する
q00 = Binary('q00')
q01 = Binary('q01')
q02 = Binary('q02')
q03 = Binary('q03')
q04 = Binary('q04')
q05 = Binary('q05')
q06 = Binary('q06')
q07 = Binary('q07')
q08 = Binary('q08')
q09 = Binary('q09')
q10 = Binary('q10')
q11 = Binary('q11')
q12 = Binary('q12')
q13 = Binary('q13')
q14 = Binary('q14')
q15 = Binary('q15')

#1番目に訪れる都市は1つだけにしたい、のやつ
H = 0
H += (q00 + q01 + q02 + q03 - 1)**2
H += (q04 + q05 + q06 + q07 - 1)**2
H += (q08 + q09 + q10 + q11 - 1)**2
H += (q12 + q13 + q14 + q15 - 1)**2

#都市Aに訪れる順番は1つだけにしたい、のやつ
H += (q00 + q04 + q08 + q12 - 1)**2
H += (q01 + q05 + q09 + q13 - 1)**2
H += (q02 + q06 + q10 + q14 - 1)**2
H += (q03 + q07 + q11 + q15 - 1)**2

#都市間の距離に比例したペナルティ
#1番目から2番目への移動について
H += 0.0 * (q00 * q04) #距離0なので
H += 0.3 * (q01 * q04) #距離3なので
H += 0.2 * (q02 * q04) #距離2なので
H += 0.6 * (q03 * q04) #距離6なので

H += 0.3 * (q00 * q05)
H += 0.0 * (q01 * q05)
H += 0.1 * (q02 * q05)
H += 0.2 * (q03 * q05)

H += 0.2 * (q00 * q06)
H += 0.1 * (q01 * q06)
H += 0.0 * (q02 * q06)
H += 0.3 * (q03 * q06)

H += 0.6 * (q00 * q07)
H += 0.2 * (q01 * q07)
H += 0.3 * (q02 * q07)
H += 0.0 * (q03 * q07)

#2番目から3番目への移動について
H += 0.0 * (q04 * q08)
H += 0.3 * (q05 * q08)
H += 0.2 * (q06 * q08)
H += 0.6 * (q07 * q08)

H += 0.3 * (q04 * q09)
H += 0.0 * (q05 * q09)
H += 0.1 * (q06 * q09)
H += 0.2 * (q07 * q09)

H += 0.2 * (q04 * q10)
H += 0.1 * (q05 * q10)
H += 0.0 * (q06 * q10)
H += 0.3 * (q07 * q10)

H += 0.6 * (q04 * q11)
H += 0.2 * (q05 * q11)
H += 0.3 * (q06 * q11)
H += 0.0 * (q07 * q11)

#3番目から4番目への移動について
H += 0.0 * (q08 * q12)
H += 0.3 * (q09 * q12)
H += 0.2 * (q10 * q12)
H += 0.6 * (q11 * q12)

H += 0.3 * (q08 * q13)
H += 0.0 * (q09 * q13)
H += 0.1 * (q10 * q13)
H += 0.2 * (q11 * q13)

H += 0.2 * (q08 * q14)
H += 0.1 * (q09 * q14)
H += 0.0 * (q10 * q14)
H += 0.3 * (q11 * q14)

H += 0.6 * (q08 * q15)
H += 0.2 * (q09 * q15)
H += 0.3 * (q10 * q15)
H += 0.0 * (q11 * q15)

#コンパイル
model = H.compile()
qubo, offset = model.to_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#上位5件
for r in result[:5]:
    print(r)

#上位2件
print(np.array(list(result[0][0].values()), int).reshape(4, 4))
print(np.array(list(result[1][0].values()), int).reshape(4, 4))

[{'q00': 1.0, 'q01': 0.0, 'q02': 0.0, 'q03': 0.0, 'q04': 0.0, 'q05': 0.0, 'q06': 1.0, 'q07': 0.0, 'q08': 0.0, 'q09': 1.0, 'q10': 0.0, 'q11': 0.0, 'q12': 0.0, 'q13': 0.0, 'q14': 0.0, 'q15': 1.0}, -7.5, 18]
[{'q00': 0.0, 'q01': 0.0, 'q02': 0.0, 'q03': 1.0, 'q04': 0.0, 'q05': 1.0, 'q06': 0.0, 'q07': 0.0, 'q08': 0.0, 'q09': 0.0, 'q10': 1.0, 'q11': 0.0, 'q12': 1.0, 'q13': 0.0, 'q14': 0.0, 'q15': 0.0}, -7.5, 9]
[{'q00': 0.0, 'q01': 0.0, 'q02': 0.0, 'q03': 1.0, 'q04': 0.0, 'q05': 0.0, 'q06': 1.0, 'q07': 0.0, 'q08': 0.0, 'q09': 1.0, 'q10': 0.0, 'q11': 0.0, 'q12': 1.0, 'q13': 0.0, 'q14': 0.0, 'q15': 0.0}, -7.3, 11]
[{'q00': 1.0, 'q01': 0.0, 'q02': 0.0, 'q03': 0.0, 'q04': 0.0, 'q05': 0.0, 'q06': 1.0, 'q07': 0.0, 'q08': 0.0, 'q09': 0.0, 'q10': 0.0, 'q11': 1.0, 'q12': 0.0, 'q13': 1.0, 'q14': 0.0, 'q15': 0.0}, -7.3, 19]
[{'q00': 0.0, 'q01': 1.0, 'q02': 0.0, 'q03': 0.0, 'q04': 0.0, 'q05': 0.0, 'q06': 0.0, 'q07': 1.0, 'q08': 0.0, 'q09': 0.0, 'q10': 1.0, 'q11': 0.0, 'q12': 1.0, 'q13': 0.0, 'q14': 0.0,